In [100]:
#Load Packages

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
import matplotlib.dates as mdates
from pypfopt import black_litterman
from pypfopt.black_litterman import BlackLittermanModel
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import black_litterman, risk_models

In [101]:
#Import data for selected companies 

df_historical = pd.read_csv(r'Historical Data.csv', index_col = 0)
pd.DataFrame(df_historical)

,JKH,SPEN,ACL,AEL,COMB
Date,,,,,
18-Dec-17,149.0,55.9,46.0,24.0,133.70
19-Dec-17,147.8,55.9,45.0,23.5,132.14
20-Dec-17,148.0,55.5,44.0,23.4,132.14
21-Dec-17,147.6,55.5,44.5,23.5,132.14
22-Dec-17,147.8,55.5,44.3,23.7,131.27
...,...,...,...,...,...
19-Nov-20,141.0,47.2,60.4,24.1,76.20
20-Nov-20,141.6,47.3,62.0,24.5,76.30
23-Nov-20,141.6,47.5,61.9,24.5,76.60


In [102]:
#Importing historical data of SL20 and bond data

df_sl20 = pd.read_csv(r'sl20.csv', index_col=[0])
df_tbonds = pd.read_csv(r't_bonds.csv', index_col=[0])

In [103]:
#Importing data of the market and treasury bonds

df_sl20 = df_sl20['Market Return']
df_tbonds = df_tbonds['Price']

In [104]:
# Calculate percentage change in sl20 market and selected stocks using log for represent data as time additive values.

prc_market = df_tbonds.pct_change().apply(lambda x: np.log(1+x))
prc_returns = df_historical.pct_change().apply(lambda x: np.log(1+x))
prc_returns

,JKH,SPEN,ACL,AEL,COMB
Date,,,,,
18-Dec-17,NaN,NaN,NaN,NaN,NaN
19-Dec-17,-0.008086,0.000000,-0.021979,-0.021053,-0.011737
20-Dec-17,0.001352,-0.007181,-0.022473,-0.004264,0.000000
21-Dec-17,-0.002706,0.000000,0.011300,0.004264,0.000000
22-Dec-17,0.001354,0.000000,-0.004505,0.008475,-0.006606
...,...,...,...,...,...
19-Nov-20,0.000000,-0.006336,-0.021295,-0.028632,0.000000
20-Nov-20,0.004246,0.002116,0.026145,0.016461,0.001311
23-Nov-20,0.000000,0.004219,-0.001614,0.000000,0.003924


In [105]:
#Calculating Covarient Matrix

cov_stocks = df_historical.cov()
cov_stocks = np.asmatrix(cov_stocks)
cov_stocks


matrix([[241.47596074,  72.07387399,  15.84109518,  11.2681103 ,
         201.80724186],
        [ 72.07387399,  36.68385045,   1.62424827,   3.93771731,
         112.18225363],
        [ 15.84109518,   1.62424827,  55.28933179,  21.1005017 ,
         -25.0358585 ],
        [ 11.2681103 ,   3.93771731,  21.1005017 ,  12.81417795,
           3.49505426],
        [201.80724186, 112.18225363, -25.0358585 ,   3.49505426,
         408.81792585]])

In [106]:
#Import market capitalization of selected companies

mrkt_cap = pd.read_excel(r'mrkt_cap.xlsx', index_col=[0])
mrkt_cap = mrkt_cap['Market Weight']
mrkt_cap = np.asmatrix(mrkt_cap)
mrkt_cap = np.transpose(mrkt_cap)
mrkt_cap

matrix([[0.54897073],
        [0.07262728],
        [0.07709902],
        [0.03739303],
        [0.26390995]])

In [107]:
#Calculating risk aversion parameter using pyportfolioopt package

delta = black_litterman.market_implied_risk_aversion(df_sl20)
delta

-0.00945210562126599

In [108]:
#Calculating prior distribution of the portfolio

prior_distribution = delta * cov_stocks * mrkt_cap
prior_distribution

matrix([[-1.82141493],
        [-0.68158358],
        [-0.06861109],
        [-0.08979705],
        [-2.12696317]])

In [109]:
#Enter the view vector and the pick matrix

Q = np.array([0.06, 0.04, 0.02, -0.01]).reshape(-1, 1)
P = np.array(
    [
        [1, 0, 0, 0, 0],
        [0, 1, 0, 0, 0],
        [0, 0, 0, 1, 0],
        [0, 0, 0, 0, 1]
    ]
)

In [110]:
#Calculate the omega value

tau = 0.25
omega = tau * P  * cov_stocks * np.transpose(P)
omega

matrix([[ 60.36899019,  18.0184685 ,   2.81702757,  50.45181046],
        [ 18.0184685 ,   9.17096261,   0.98442933,  28.04556341],
        [  2.81702757,   0.98442933,   3.20354449,   0.87376357],
        [ 50.45181046,  28.04556341,   0.87376357, 102.20448146]])

In [111]:
#Calculating the inverse of the omega matrix

omega_inv = np.diag(1 / np.diag(omega))
omega_inv

array([[0.0165648 , 0.        , 0.        , 0.        ],
       [0.        , 0.10903981, 0.        , 0.        ],
       [0.        , 0.        , 0.31215424, 0.        ],
       [0.        , 0.        , 0.        , 0.00978431]])

In [112]:
tau = 0.25
sigma_inv = (np.linalg.inv(cov_stocks))
sigma_inv

matrix([[ 0.01062592, -0.02841055, -0.00166587,  0.00146516,  0.00243616],
        [-0.02841055,  0.29704761, -0.03835786,  0.01594889, -0.06997266],
        [-0.00166587, -0.03835786,  0.06343729, -0.09558481,  0.01605002],
        [ 0.00146516,  0.01594889, -0.09558481,  0.23277445, -0.01294334],
        [ 0.00243616, -0.06997266,  0.01605002, -0.01294334,  0.021538  ]])

In [113]:
A = tau * sigma_inv * prior_distribution 
A

matrix([[-1.29723233e-03],
        [-1.71620175e-04],
        [-1.82187022e-04],
        [-8.83607057e-05],
        [-6.23626177e-04]])

In [114]:
p_omega_inv = P.T @ omega_inv
tau_sigma_inv = tau * sigma_inv

A = tau_sigma_inv + p_omega_inv @ P
b = (tau_sigma_inv @ prior_distribution + p_omega_inv @ Q)
expected_returns = np.linalg.solve(A, b)
expected_returns

matrix([[-0.00654291],
        [ 0.02140312],
        [ 0.03629499],
        [ 0.01848966],
        [-0.02827384]])

In [115]:
#Calculate the weights for the asstes of the portfolio

C = delta * cov_stocks
C = np.linalg.inv(C)
weights = np.linalg.solve(C, expected_returns)
weights

matrix([[ 0.04688167],
        [ 0.02577105],
        [-0.02869512],
        [-0.00864402],
        [ 0.10701921]])